<a href="https://colab.research.google.com/github/encoras/Artificial-Intelligence-Group/blob/master/cnn_amber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, BatchNormalization
from keras.layers import Dense, Dropout
from keras.layers import Flatten

In [3]:
img_rows = 50
img_cols = 50
epochs = 400
batch_size = 5*32
num_of_train_samples = 1265
num_of_test_samples = 505
num_of_val_samples = 200

In [4]:
#part2-fitting the cnn to the images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True)

In [5]:
# Generating images for the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
# Creating training set
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Amber_yolo/train',
                                                 target_size = (img_rows, img_cols),
                                                 batch_size = batch_size,
                                                 shuffle = True,
                                                 class_mode = 'categorical')
# Creating the Test set
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Amber_yolo/test',
                                            target_size = (img_rows, img_cols),
                                            batch_size = batch_size,
                                            shuffle = False,
                                            class_mode = 'categorical')

# Creating the Test set
valid_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Amber_yolo/val',
                                            target_size = (img_rows, img_cols),
                                            batch_size = batch_size,
                                            shuffle = False,
                                            class_mode = 'categorical')

Found 1265 images belonging to 20 classes.
Found 504 images belonging to 20 classes.
Found 200 images belonging to 20 classes.


In [6]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
# Making Sppnet custom layer
# Try refering github implementation
# Refer https://keras.io/layers/writing-your-own-keras-layers/
# or https://keras.io/examples/antirectifier/
import sys
from keras.layers import Layer
from keras import backend as K

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

BATCH = 20

class SppnetLayer(Layer):
    '''This layer takes an input tensor and pools the tensor
      in local spatial bins.
      This layer uses Max pooling.
      It accepts input in tensorflow format. # channels last

    # Input
        list of filter in form [x,y,z] 
    # Input shape : 4d tensor [None, X,Y, channels]
    # Output shape : 3d tensor [None,pooled dim, channels] 

    '''
    def __init__(self, filters = [1], **kwargs):
        self.filters = filters
        super(SppnetLayer, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        length = 0;
        for f_size in self.filters:
            length+= (f_size*f_size)
        return (input_shape[0],length*input_shape[3])
      
    def get_config(self):
        config = {'filters': self.filters}
        base_config = super(SppnetLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs):
      output = []
      for f_size in self.filters:
        win_size = K.int_shape(inputs)[1]/f_size
        #print(win_size)
        win_size = int(win_size)
        for x_start in range(0,f_size):
          for y_start in range(0,f_size):
            X = int(x_start*win_size)
            Y = int(y_start*win_size)
            result = K.max(inputs[:,X:X+win_size,Y:Y+win_size,:],axis = (1,2))
            output.append(result)
      output = K.concatenate(output)
      return output

In [8]:
model=Sequential()

model.add(Conv2D(8,(5,5),activation='relu',input_shape=(img_rows,img_cols,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.15))

model.add(Conv2D(16,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.15))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.15))

#model.add(SppnetLayer([1,2,4])) #To implement SppNet 
model.add(Flatten())
model.add(Dense(320,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Dense(20,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy', f1_m])

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 8)         608       
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 8)        32        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 8)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 8)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 16)        1168      
                                                                 
 batch_normalization_1 (Batc  (None, 21, 21, 16)       6

In [9]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',patience = 2,verbose = 1,factor = 0.8,min_lr = 0.00001)
mcp_save = ModelCheckpoint('/content/drive/MyDrive/Amber_yolo/mdl_wts1.hdf5', save_best_only=True, monitor='val_f1_m', mode='max')
callbacks = [mcp_save] #[earlystop,learning_rate_reduction]

In [ ]:
history = model.fit_generator(training_set,steps_per_epoch=num_of_train_samples // batch_size,epochs=epochs, validation_data=test_set, validation_steps=num_of_test_samples // batch_size, callbacks=callbacks)

<ipython-input-10-79c0ae024095>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(training_set,steps_per_epoch=num_of_train_samples // batch_size,epochs=epochs, validation_data=test_set, validation_steps=num_of_test_samples // batch_size, callbacks=callbacks)


Epoch 1/400
7/7 [==============================] - 245s 37s/step - loss: 2.9627 - accuracy: 0.2027 - f1_m: 0.1434 - val_loss: 2.9544 - val_accuracy: 0.0917 - val_f1_m: 0.0000e+00
Epoch 2/400
7/7 [==============================] - 8s 1s/step - loss: 2.0846 - accuracy: 0.3367 - f1_m: 0.2635 - val_loss: 2.9483 - val_accuracy: 0.0542 - val_f1_m: 0.0000e+00
Epoch 3/400
7/7 [==============================] - 10s 1s/step - loss: 1.8141 - accuracy: 0.3910 - f1_m: 0.3044 - val_loss: 2.9788 - val_accuracy: 0.0896 - val_f1_m: 0.0000e+00
Epoch 4/400
7/7 [==============================] - 9s 1s/step - loss: 1.6600 - accuracy: 0.4145 - f1_m: 0.3586 - val_loss: 3.0392 - val_accuracy: 0.0604 - val_f1_m: 0.0000e+00
Epoch 5/400
7/7 [==============================] - 9s 1s/step - loss: 1.5488 - accuracy: 0.4624 - f1_m: 0.3723 - val_loss: 3.1750 - val_accuracy: 0.0958 - val_f1_m: 0.0000e+00
Epoch 6/400
7/7 [==============================] - 9s 1s/step - loss: 1.4379 - accuracy: 0.4905 - f1_m: 0.4220 - val

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
#AlexNet Inmplementation with batch normalization layers
model1 = Sequential()
model1.add(Conv2D(filters=96, input_shape=(img_rows,img_cols,3), kernel_size=(11,11), strides=(4,4), padding='same',activation = 'relu'))
model1.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model1.add(BatchNormalization())
model1.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same',activation = 'relu'))
model1.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model1.add(BatchNormalization())
model1.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same',activation = 'relu'))
model1.add(BatchNormalization())
model1.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same',activation = 'relu'))
model1.add(BatchNormalization())
model1.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same',activation = 'relu'))
model1.add(BatchNormalization())
model1.add(Flatten()) # To implement Simple Alexnet

model1.add(Dense(196,activation = 'relu'))
model1.add(Dropout(0.2))
model1.add(Dense(196,activation = 'relu'))
model1.add(Dropout(0.2))
model1.add(Dense(20,activation= 'softmax'))
model1.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy', f1_m])
print(model1.summary())

In [ ]:
mcp_save = ModelCheckpoint('/content/drive/MyDrive/Amber_yolo/mdl_wts2.hdf5', save_best_only=True, monitor='val_f1_m', mode='max')
callbacks = [mcp_save] #[earlystop,learning_rate_reduction]
history1 = model1.fit_generator(training_set,epochs=epochs, validation_data=test_set,callbacks=callbacks)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history1.history['loss'], color='b', label="Training loss")
ax1.plot(history1.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history1.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history1.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
model.save("/content/drive/MyDrive/Amber_yolo/model1_amber150epoch.h5")

In [ ]:
model.load_weights('/content/drive/MyDrive/Amber_yolo/mdl_wts1.hdf5')
y_pred = model.predict_generator(test_set, steps = 505 // batch_size+1, verbose=1)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = np.argmax(y_pred, axis=1)
y_pred

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['N01', 'N02', 'N03','N04','N05','N06','N07','N08','N09','N10','N11','N12','N13','N14','N15','N16','N17','N18','N19','N20']
print(classification_report(test_set.classes, y_pred, target_names=target_names))

In [ ]:
model1.load_weights('/content/drive/MyDrive/Amber_yolo/mdl_wts2.hdf5')
y_pred = model1.predict_generator(test_set, steps = 505 // batch_size+1)
y_pred = np.argmax(y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_set.classes, y_pred))
print('Classification Report')
target_names = ['N01', 'N02', 'N03','N04','N05','N06','N07','N08','N09','N10','N11','N12','N13','N14','N15','N16','N17','N18','N19','N20']
print(classification_report(test_set.classes, y_pred, target_names=target_names))